# NumPy Intro

*version 0.2 by ozi & jan*

Python is an interpreted language and as such it is extremely flexible, allowing to define everything, including code itself, 
at runtime. This entails that the Python interpreter uses a lot of magic behind the scenes (e.g. type inferencing) to keep things
as simple and productive as possible for the programmer. This flexibility comes at the price of a markedly reduced runtime speed 
compared to compiled languages such as C++.
Many problems encoded in Python do not require the full range of flexibility and could easily trade it for improved runtime 
performance without giving up the development performance which is the key for Python's success.

NumPy is a library, written in C, to enable fast numerical computing in Python. 
The main sacrifice to enable this speed is the restriction to 
containers with uniform memory layout, i.e., arrays that contain items of uniform static datatype.

Despite this apparently severe restriction NumPy is quite versatile and giving a full introduction into NumPy would require a
course of its own. Hence we will focus on only few key features and an overview of the available functionality and leave in-depth
exploration to the user. The numpy website http://numpy.scipy.org is a good starting point.

**Note:** NumPy is not a part of the vanilla Python distribution and needs to be installed separately. Then it can be used  like any other Python library:

In [2]:
import numpy as np
np.__version__

'1.11.0'

## ndarray

The main datatype in NumPy is an n-dimensional array object called `ndarray`. By default it has C-array memory layout 
(last index fastest, 0 based indices) and its item type is `float64`. If the `ndarray` is created from user-specified data the default item type will be the 'minimal' datatype that can hold the data unless explicitely specified otherwise.

### creation, type

In [3]:
a=np.array([[1,2,3,4],[5,6,7,8]])  # from python list
b=np.zeros(5)  # float64 if no dtype given!
c=np.arange(10,10+10j) # from generator
d=np.linspace(0,2*np.pi, 360) # dto.
e=np.ones(10,dtype='bool') # explicit specification
for i in [a,b,c,d,e]:
    print(i.dtype)

int64
float64
complex128
float64
bool


### inspection, shape

Several meta data of an `ndarray` can be obtained. Try also `a.ndim,a.size,a.nbytes,a.flags,type(a)`.
What is the size of a `bool`?

In [4]:
a=np.array([[1,2,3],[4,5,6]])
print(a)  # data
print(a.dtype) # data type
print(a.itemsize) # element size in bytes
print(a.shape) # array dimensions
print("Add a.ndim, ... ")
b = np.array([1], dtype=bool)
print(b.nbytes)

[[1 2 3]
 [4 5 6]]
int64
8
(2, 3)
Add a.ndim, ... 
1


## dtypes

I you use large arrays, memory may become an issue. NumPy provides data types in various sizes, choose wisely!

**NumPy dtypes**
* (long)float, float16...128
* (long)complex, complex64...256
* (u)int, (u)int8...128
* str, unicode
* bool (=1byte!)
* datetime64,timedelta64
* object, void

### complex numbers

NumPy supports arrays with type complex
all `ndarray`s have methods **`real()`** and **`imag()`**
* ** `conj()`**  	return the complex conjugate, element-wise
* **`angle()`** 	return the angle of the complex argument

In [5]:
a = np.array([1+1j, 2, 3-2j])
a.conj()

array([ 1.-1.j,  2.-0.j,  3.+2.j])

In [6]:
np.angle(a,deg=1) # default: radian 

array([ 45.        ,   0.        , -33.69006753])

### casting

In [7]:
a=np.array([[129,128,127],[226,125,124.9]],dtype='int8') # type coercion - no warnings, no exceptions!
a

array([[-127, -128,  127],
       [ -30,  125,  124]], dtype=int8)

**type-casting** 
* `astype()`
* `asarray()`

**related functions**
* `tolist()`
* `fromstring()`

In [8]:
a.astype('complex64')

array([[-127.+0.j, -128.+0.j,  127.+0.j],
       [ -30.+0.j,  125.+0.j,  124.+0.j]], dtype=complex64)

In [9]:
arr_address=hex(a.__array_interface__['data'][0])
arr_address

'0x2ccceb0'

almost anything can be cast into an ndarray, whether it makes sense or not:

In [10]:
a=[None,None,1]
b=("blahh",12)
c=((((1+9j,1-2j),(12,-100.0))))
d=[type,np.array,list]
for ii,i in enumerate([a,b,c,d]):
 x=np.asarray(i)
 print("%d: data="%(ii+1),x, ", dtype=%s"%(x.dtype))

1: data= [None None 1] , dtype=object
2: data= ['blahh' '12'] , dtype=<U5
3: data= [[   1.+9.j    1.-2.j]
 [  12.+0.j -100.+0.j]] , dtype=complex128
4: data= [<class 'type'> <built-in function array> <class 'list'>] , dtype=object


Note that most functionality is geared towards numerical arrays. If you often need 'record type' arrays you may want to have a loook at [pandas](http://pandas.pydata.org/pandas-docs/stable/).

## ndarrays: read access

Tuple indexing and slicing of an `ndarray` works in the same way as with normal python lists. The only difference is that tuple indexing of multidimensional ndarrays uses a single square bracket with comma-separated indices. 
Remember: in Python slices are references!

In [11]:
a=np.array([[1,2,3],[4,5,6],[7,8,9]])
a[1,-1]   # tuple indexing in 1 square bracket

6

In [12]:
a[:,::2] # every second column

array([[1, 3],
       [4, 6],
       [7, 9]])

In [ ]:
a[1:3]

But an `ndarray` can be accessed also in more sophisticated ways unavailable for python lists:

**fancy indexing**

In [13]:
a=np.arange(0,77,7)
a[[1,3,8]]

array([ 7, 21, 56])

**using a condition **

In [14]:
a[a%3==0]

array([ 0, 21, 42, 63])

**using a boolean mask**

In [15]:
mask=np.array([0,1,0,1,0,0,0,0,1,0,0],dtype=bool)
a[mask]

array([ 7, 21, 56])

**using an index array**

Index arrays can be created e.g. by the `where()` function:

In [16]:
div_by_three=np.where(a%3==0)
print(div_by_three)

(array([0, 3, 6, 9]),)


...and then be used for fancy indexing:

In [17]:
a[div_by_three]

array([ 0, 21, 42, 63])

A typical use case for this is accessing a second array (of the same dimensions) based on a condition on the first array.

## ndarrays: write access

As slices (including single cells) are references, values can be directly assigned to them.
Alternatively they can be filled with `fill()`.

In [18]:
a=np.array([[1,2,3],[4,5,6]])
a[1,1:]=0
a

array([[1, 2, 3],
       [4, 0, 0]])

In [19]:
a.fill(2.5)   # same as a[:]=2.5
print(a) 
print(a.dtype)

[[2 2 2]
 [2 2 2]]
int64


Oops! This is NumPy not Python: type coercion, again!

## ndarrays: shape shifting

For efficient handling of multidimensional arrays it is important to distinguish between memory layout and access patterns.
The current access pattern is stored in the `strides` property. NumPy provides a range of methods to manipulate views on `ndarrays`:
<table>
<tr><td> `a.transpose()` or `a.T`</td><td> inverts the strides</td></tr>
<tr><td> `a.flatten()`</td><td>            creates a 1d copy </td></tr>
<tr><td> `a.flat`</td><td>                 is a 1d iterator on `a`</td></tr>
<tr><td> `a.ravel()`</td><td>              creates a view. A view is a reference *iff* its source is contiguous in memory.</td></tr>
<tr><td> `np.newaxis`</td><td>             a special index to add an empty dimension</td></tr>
<tr><td> `a.squeeze()`</td><td>               removes all empty dimensions</td></tr>
<tr><td> `a.reshape`</td><td>             explicit shape definition, max one dimension can be guessed (enter `-1`)</td></tr>
<tr><td> `a.swapaxes()`</td><td>               swaps two axis of the `ndarray`</td></tr>
</table>

In [20]:
a=np.array([[1,2,3],[4,5,6]])
a.strides

(24, 8)

In [21]:
b=a.T
b[0,0]=0
print(b)
print(b.strides)

[[0 4]
 [2 5]
 [3 6]]
(8, 24)


In [22]:
a # a view is a reference, strides may be changed

array([[0, 2, 3],
       [4, 5, 6]])

In [23]:
a=np.array([[1,2,3],[4,5,6]])
c=a.flatten()
c

array([1, 2, 3, 4, 5, 6])

In [24]:
sum(a.flat)

21

In [25]:
c=a.ravel() # reference!
c[0]=0
a[0]

array([0, 2, 3])

In [26]:
a=np.array([[1,2,3],[4,5,6]])
a_t=a.T # a_t is no longer contiguous in memory
d=a_t.ravel() # copy!
d[0]=0
a_t[0]

array([1, 4])

In [27]:
a=np.array([1,2,3]) # 1d-array, shape=(3, )
b=a[:,np.newaxis,np.newaxis]
b  #3d-array, shape==(3,1,1)

array([[[1]],

       [[2]],

       [[3]]])

In [30]:
c=a[np.newaxis,np.newaxis,:]
c #3d-array, shape==(1,1,3)

array([[[1, 2, 3]]])

In [31]:
b.squeeze()

array([1, 2, 3])

In [32]:
d=np.arange(27).reshape(3,-1,3); d

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [33]:
d.swapaxes(0,2)

array([[[ 0,  9, 18],
        [ 3, 12, 21],
        [ 6, 15, 24]],

       [[ 1, 10, 19],
        [ 4, 13, 22],
        [ 7, 16, 25]],

       [[ 2, 11, 20],
        [ 5, 14, 23],
        [ 8, 17, 26]]])

## ndarrays: more on creation

NumPy has several convenience functions to create ndarrays with data like:

<table>
<tr><td> `zeros()`</td><td>		0-filled array</td></tr>
<tr><td> `identity()`</td><td>	square 0-filled array, main diagonal=1</td></tr>
<tr><td> `asarray()`</td><td>	from list, tuple and their combinations</td></tr>
<tr><td> `fromiter()`</td><td>	from iterator</td></tr>
<tr><td> `fromfunction()`</td><td> from function evaluated on array positions</td></tr>
<tr><td> `diag()`</td><td>		like identity but diagonal given as vector </td></tr>
</table>

Often the data to fill ndarrays reside in files. `loadtext()` provides minimal file parsing for normal and gzip'd text files.
Its counterpart is `savetxt()`.

In [35]:
cat 'test.dat'

////       This and the next line are headers
Name            income          rev_q1  rev_q2  rev_q3  rev_q4
larry,          200,            0,      1.5,    2.8,    0.2
bill,           1000,           12.5,   15,     18,     33.4
steve,          5600,           128,    40,     0,      0


In [36]:
np.loadtxt('test.dat',dtype=np.float32, skiprows=2, delimiter=',' ,usecols=[1,2,3,4,5])

array([[  2.00000000e+02,   0.00000000e+00,   1.50000000e+00,
          2.79999995e+00,   2.00000003e-01],
       [  1.00000000e+03,   1.25000000e+01,   1.50000000e+01,
          1.80000000e+01,   3.34000015e+01],
       [  5.60000000e+03,   1.28000000e+02,   4.00000000e+01,
          0.00000000e+00,   0.00000000e+00]], dtype=float32)

In [37]:
dt=np.dtype([('name','S10'),('income',np.int16),\
             ('revenue',([('q1',np.float32),('q2',np.float32),('q3',np.float32),('q4',np.float32)]))])
np.loadtxt('test.dat',dtype=dt, skiprows=2,delimiter=',')

array([(b'larry', 200, (0.0, 1.5, 2.799999952316284, 0.20000000298023224)),
       (b'bill', 1000, (12.5, 15.0, 18.0, 33.400001525878906)),
       (b'steve', 5600, (128.0, 40.0, 0.0, 0.0))], 
      dtype=[('name', 'S10'), ('income', '<i2'), ('revenue', [('q1', '<f4'), ('q2', '<f4'), ('q3', '<f4'), ('q4', '<f4')])])

In [38]:
a=np.arange(0,40,0.5)
np.savetxt("mydata.txt",a.reshape(-1,5),fmt="%5.1f",delimiter=",")  # reshaping into 5 columns
!cat 'mydata.txt'

  0.0,  0.5,  1.0,  1.5,  2.0
  2.5,  3.0,  3.5,  4.0,  4.5
  5.0,  5.5,  6.0,  6.5,  7.0
  7.5,  8.0,  8.5,  9.0,  9.5
 10.0, 10.5, 11.0, 11.5, 12.0
 12.5, 13.0, 13.5, 14.0, 14.5
 15.0, 15.5, 16.0, 16.5, 17.0
 17.5, 18.0, 18.5, 19.0, 19.5
 20.0, 20.5, 21.0, 21.5, 22.0
 22.5, 23.0, 23.5, 24.0, 24.5
 25.0, 25.5, 26.0, 26.5, 27.0
 27.5, 28.0, 28.5, 29.0, 29.5
 30.0, 30.5, 31.0, 31.5, 32.0
 32.5, 33.0, 33.5, 34.0, 34.5
 35.0, 35.5, 36.0, 36.5, 37.0
 37.5, 38.0, 38.5, 39.0, 39.5


NumPy also provides file I/O functions for binary formats, including NumPy's own proprietary formats:

* **`fromfile(), tofile()`** for binary or text format
* **`save(), load()`** for binary NumPy "npy"-format
* **`savez(), load()`** for binary NumPy "npz"-container format
* **`genfromtxt()`** like `loadtxt()` but w. more options, checks etc.

For "big data", however, you should have a look on external python libraries to access HDF5 or netCDF files.

## Working with ndarrays: elementwise operations

Elementwise operations of NumPy are one of the main reasons why NumPy is much faster than Python. They can often replace loops (or list comprehensions) and thereby vectorize your code.

In [39]:
l = range(1000000)  # python
%timeit [i+1 for i in l]  

10 loops, best of 3: 79.1 ms per loop


In [40]:
a = np.arange(1000000) # NumPy
%timeit a + 1

1000 loops, best of 3: 1.31 ms per loop


## Working with ndarrays: ufuncs

For more complex arithmetic operations NumPy provides `ufunc`s. These are vectorized wrappers for scalar functions, i.e. they also process arrays elementwise. There are about 80 ufuncs available. http://docs.scipy.org/doc/numpy/reference/ufuncs.html#available-ufuncs

In [41]:
from math import pow as pypow, sinh as pysinh
a=np.random.random(1000000)
b=np.random.random(1000000)
c=a.tolist()
d=b.tolist()

**Python:**

In [42]:
%timeit [pysinh(i) for i in c]

10 loops, best of 3: 113 ms per loop


In [43]:
%timeit [pypow(i,j) for i,j in zip(c,d)]

1 loop, best of 3: 211 ms per loop


**NumPy:**

In [44]:
%timeit np.sinh(a)

10 loops, best of 3: 24 ms per loop


In [45]:
%timeit np.power(a,b)

10 loops, best of 3: 63.7 ms per loop


### ufuncs: broadcasting

Working with two or more arrays typically require these to have the same number of dimensions. To avoid having to use  `np.newaxis` where it can be inferred, there is an implicit mechanism for upcasting the dimensionality of an `ndarray` called broadcasting. For a nice illustration what happens in broadcasting, look [here](https://scipy-lectures.github.io/intro/numpy/operations.html#broadcasting).

In [48]:
a=np.array([[4.,2.,6.],[1.,2.,3.]])
b=np.array([1.,5.,3.]) # will be broadcasted to 2d-array: array([[1.,5.,3.],[1.,5.,3.]])
np.maximum(a,b)

array([[ 4.,  5.,  6.],
       [ 1.,  5.,  3.]])

* the output shape is the maximum shape for each of the dimensions occurring in the inputs
* values are repeated from the values of axis 

### ufuncs: collective methods

`ufunc`s support the following collective methods:
* **_op_`.reduce()`**     apply `ufunc` _op_ along one axis (output dim=input dim -1)
* **_op_`.accumulate()`** same but with intermediate results
* **_op_`.reduceat()`** reduce on specific slices
* **_op_`.outer()`** combinatoric application of `ufunc` _op_

In [49]:
a=np.array([[1.,2.,3.],[4.,5.,6.]])
np.add.reduce(a)

array([ 5.,  7.,  9.])

In [50]:
np.add.reduce(a,axis=1)

array([  6.,  15.])

In [51]:
np.add.accumulate(a, axis=1)

array([[  1.,   3.,   6.],
       [  4.,   9.,  15.]])

## vectorize and vectorize

In [52]:
a=np.random.random(1000000)
al=a.tolist()

In [53]:
%timeit np.sinh(a)

10 loops, best of 3: 24.1 ms per loop


In [54]:
from math import pow as pypow, sinh as pysinh

In [55]:
%timeit [pysinh(i) for i in al]

10 loops, best of 3: 112 ms per loop


In [56]:
b=np.random.random(1000000)
bl=b.tolist()

In [57]:
%timeit [pypow(i,j) for i,j in zip(al,bl)]

1 loop, best of 3: 206 ms per loop


In [58]:
%timeit np.power(a,b)

10 loops, best of 3: 63.3 ms per loop


[Numba](http://numba.pydata.org/) offers a decorator `@vectorize` that allows us to generate fast ufuncs. The decorator takes a list of type specifications of the form `f8(f8, f8)`, where the type before the parentheses is the return type and the types within the parentheses are the argument types.

In [59]:
from numba import vectorize
import math

In [60]:
@vectorize(['f8(f8)','f4(f4)'])
def mysinh(x):
    return pysinh(x)

In [61]:
%timeit mysinh(a)

10 loops, best of 3: 23.9 ms per loop


Use vectorize to make your favorite function into a ufunc.

In [62]:
def my_function(i):
    if i!=0:
        return 2.718281**(-1./(i*i))
    else:
        return 0

In [63]:
al=a.tolist()
%timeit [my_function(x) for x in al]

1 loop, best of 3: 301 ms per loop


There is also a vectorize function in numpy:

In [64]:
v_func=np.vectorize(my_function)

%timeit v_func(a)

1 loop, best of 3: 355 ms per loop


In [65]:
nb_my_function = vectorize(['f8(f8)','f4(f4)'])(my_function)

%timeit nb_my_function(a)

10 loops, best of 3: 60.2 ms per loop


Don't confuse np.vectorize (which is only a convenient wrapper) with Numba's vectorize function:

## ... to be continued

In [47]:
np.__config__.show()

lapack_opt_info:
    libraries = ['mkl_lapack95_lp64', 'mkl_intel_lp64', 'mkl_intel_thread', 'mkl_core', 'iomp5', 'pthread']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    library_dirs = ['/home/scpy/miniconda/envs/hpcpy3/lib']
    include_dirs = ['/home/scpy/miniconda/envs/hpcpy3/include']
openblas_lapack_info:
  NOT AVAILABLE
lapack_mkl_info:
    libraries = ['mkl_lapack95_lp64', 'mkl_intel_lp64', 'mkl_intel_thread', 'mkl_core', 'iomp5', 'pthread']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    library_dirs = ['/home/scpy/miniconda/envs/hpcpy3/lib']
    include_dirs = ['/home/scpy/miniconda/envs/hpcpy3/include']
blas_mkl_info:
    libraries = ['mkl_intel_lp64', 'mkl_intel_thread', 'mkl_core', 'iomp5', 'pthread']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    library_dirs = ['/home/scpy/miniconda/envs/hpcpy3/lib']
    include_dirs = ['/home/scpy/miniconda/envs/hpcpy3/include']
mkl_info:
    libraries = ['mkl_intel_lp